# 2 文本数据处理

**本章内容**

- 为大型语言模型训练准备文本
- 将文本分割为词汇和子词汇token
- 字节对编码是一种更高级的文本分词方法
- 使用滑动窗口方法抽样训练示例
- 将token转换为向量输入大型语言模型

在前一章中，我们探讨了大型语言模型（LLM）的一般结构，并了解到它们是在大量文本上进行预训练的。
具体来说，我们关注的是基于变transomer架构的仅解码器模式的大型语言模型(LLMs)，这种架构是ChatGPT以及其他流行的类似GPT的LLM的基础。

在预训练阶段，大型语言模型（LLMs）逐词处理文本。
使用下一个词预测任务训练具有数百万到数十亿参数的大型语言模型，能够产生卓越能力的模型。
这些模型可以进一步微调，以遵循一般指令或执行特定的目标任务。
但在在接下来的章节中，部署和训练大型语言模型（LLMs）之前，我们需要先准备训练数据集，这也是本章的主要内容，如图2.1所示。

**图 2.1 展示了构建大型语言模型（LLM）的三个主要阶段：在通用文本数据集上预训练LLM，以及在标注数据集上对其进行微调。本章将解释并构建为LLM提供预训练文本数据的数据准备和采样流程。**

![fig2.1](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-2-1.jpg?raw=true)

在本章中，您将学习如何为训练大型语言模型（LLMs）准备输入文本。
这包括将文本分割成单个词汇和子词汇token，然后将它们编码成向量表示，供大型语言模型（LLM）使用。
您还将了解字节对编码等高级分词方案，这些方案已经在GPT等流行的大型语言模型（LLMs）中得到应用。
最后，我们将介绍采样和数据加载策略，这些策略用于生成后续章节中训练大型语言模型（LLMs)所需的输入输出对。

In [6]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import tiktoken

def create_tokenization_visualization():
    # 创建一系列帧
    frames = []
    width = 800
    height = 400
    
    # 加载中文字体（请确保这个路径指向系统中的一个中文字体文件）
    font_path = "C:\\Windows\\Fonts\\simhei.ttf"  # Windows系统默认黑体字体路径
    font_large = ImageFont.truetype(font_path, 32)
    font_small = ImageFont.truetype(font_path, 24)
    
    # 示例文本
    text = "人工智能正在改变世界"
    
    # 使用tiktoken进行分词
    enc = tiktoken.get_encoding("gpt2")
    tokens = enc.encode(text)
    
    # 为每一帧创建图像
    for i in range(len(text) + len(tokens)):
        # 创建新的空白图像
        img = Image.new('RGB', (width, height), 'white')
        draw = ImageDraw.Draw(img)
        
        # 绘制标题
        draw.text((width//2-100, 20), "分词和向量表示过程", font=font_large, fill='black')
        
        if i < len(text):
            # 显示逐字分词
            highlighted_text = text[:i+1]
            draw.text((50, 100), highlighted_text, font=font_large, fill='black')
            draw.text((50, 200), f"Token IDs: {tokens[:i+1]}", font=font_small, fill='blue')
        else:
            # 显示向量表示
            token_idx = i - len(text)
            vector = np.random.randn(10)
            draw.text((50, 100), text, font=font_large, fill='black')
            draw.text((50, 200), f"Token IDs: {tokens}", font=font_small, fill='blue')
            draw.text((50, 300), f"Vector {token_idx}: [{', '.join([f'{x:.2f}' for x in vector])}]", 
                     font=font_small, fill='green')
        
        frames.append(img)
    
    # 保存为GIF
    frames[0].save(
        'tokenization.gif',
        save_all=True,
        append_images=frames[1:],
        duration=1000,
        loop=0
    )

# 创建可视化
create_tokenization_visualization()

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.font_manager import FontProperties
import tiktoken
from IPython.display import Image, HTML
from matplotlib.patches import Rectangle, FancyBboxPatch

def create_tokenization_animation():
    # 设置中文字体
    try:
        font = FontProperties(fname=r"C:\Windows\Fonts\SimHei.ttf")
    except:
        font = FontProperties(family='SimHei')
    
    # 创建示例中文句子
    text = "人工智能正在改变世界"
    
    # 使用tiktoken进行分词
    enc = tiktoken.get_encoding("gpt2")
    tokens = enc.encode(text)
    
    # 创建图像
    plt.rcParams['figure.facecolor'] = '#f0f0f0'
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['axes.grid'] = False
    
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.set_facecolor('white')
    
    def visualize_vector(vector, x, y, width=4, height=1):
        """将向量可视化为热力图"""
        normalized = (vector - vector.min()) / (vector.max() - vector.min())
        for i, value in enumerate(normalized):
            color = plt.cm.viridis(value)
            rect = Rectangle((x + i * width/len(vector), y), 
                           width/len(vector), height, 
                           facecolor=color, edgecolor='white')
            ax.add_patch(rect)
            ax.text(x + i * width/len(vector) + width/(2*len(vector)), y + height/2, 
                   f'{vector[i]:.2f}', ha='center', va='center', 
                   fontsize=8, color='white' if value < 0.5 else 'black')
    
    def animate(i):
        ax.clear()
        ax.set_xlim(0, 16)
        ax.set_ylim(0, 9)
        
        # 添加卡片效果（使用FancyBboxPatch）
        card = FancyBboxPatch(
            (0.5, 0.5), 15, 8,
            boxstyle="round,pad=0.5",
            facecolor='white',
            edgecolor='#dddddd',
            linewidth=2
        )
        ax.add_patch(card)
        
        # 添加标题
        ax.text(8, 8, "大语言模型的文本处理过程", 
               fontsize=24, fontproperties=font,
               ha='center', va='center', color='#2c3e50',
               bbox=dict(facecolor='white', edgecolor='none', alpha=0.8))
        
        # 显示原始文本
        ax.text(0.8, 7, "输入文本:", fontsize=14, fontproperties=font, color='#7f8c8d')
        ax.text(2.3, 7, text, fontsize=16, fontproperties=font, color='#2c3e50')
        
        if i < len(text):
            # 第一阶段：分词过程
            highlighted_text = text[:i+1]
            remaining_text = text[i+1:]
            
            ax.text(0.8, 5.5, "步骤 1: 文本分词", fontsize=18, fontproperties=font, color='#3498db')
            
            # 使用背景色突出显示当前处理的字符
            ax.text(0.8, 4.5, highlighted_text, fontsize=20, fontproperties=font, color='#e74c3c')
            ax.text(0.8 + len(highlighted_text)/2, 4.5, remaining_text, fontsize=20, fontproperties=font, color='#95a5a6')
            
            # 显示Token IDs
            current_tokens = tokens[:i+1]
            ax.text(0.8, 3.5, f"Token IDs: {current_tokens}", fontsize=14, color='#34495e')
            
        elif i < len(text) + len(tokens):
            # 第二阶段：向量表示
            token_idx = i - len(text)
            current_token = tokens[token_idx]
            
            ax.text(0.8, 5.5, "步骤 2: 向量表示", fontsize=18, fontproperties=font, color='#3498db')
            ax.text(0.8, 4.5, f"Token ID {current_token} 的向量表示:", fontsize=16, color='#34495e')
            
            # 生成模拟向量并可视化
            vector = np.random.randn(10)
            visualize_vector(vector, 0.8, 3.5)
            
            # 添加解释
            ax.text(0.8, 2.5, "说明: 每个彩色块代表向量中的一个维度，颜色深浅表示数值大小", 
                   fontsize=12, color='#7f8c8d')
        
        # 添加进度条
        total_steps = len(text) + len(tokens)
        progress = (i + 1) / total_steps
        progress_bar = Rectangle((0.8, 1.5), 14.4 * progress, 0.2, facecolor='#3498db')
        progress_outline = Rectangle((0.8, 1.5), 14.4, 0.2, facecolor='none', edgecolor='#bdc3c7')
        ax.add_patch(progress_bar)
        ax.add_patch(progress_outline)
        ax.text(8, 1.2, f'进度: {i+1}/{total_steps}', ha='center', fontsize=12, color='#7f8c8d')
        
        ax.axis('off')
        return []  # 返回空列表，因为我们不使用blitting
    
    # 创建动画
    anim = animation.FuncAnimation(
        fig, 
        animate,
        frames=len(text) + len(tokens),
        interval=2000,
        blit=False,  # 关闭blitting
        repeat=True
    )
    
    # 保存为HTML5视频，而不是GIF
    html = anim.to_jshtml()
    plt.close()
    
    return HTML(html)

# 运行动画
create_tokenization_animation()